<a href="https://colab.research.google.com/github/divsal009/div/blob/master/Firefly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import glob
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions

%matplotlib inline


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import random
import math
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn import svm
import numpy as np


class FireflyAlgorithm():

    def __init__(self, function):
        self.D = X.shape[1]  # dimension of the problem (Gene number)
        self.NP = 100  # population size (Firefly number)
        self.nFES = 1  # number of function evaluations (repeate number)
        self.alpha = 1  # alpha parameter,(randomization parameter)
        self.betamin = 0.5  # beta parameter
        self.gamma = 1  # gamma parameter (light intensity coefficency)
        # sort of fireflies according to fitness value
        self.Index = [0] * self.NP
        self.Fireflies = [[np.random.rand() for i in range(self.D)] for j in range(self.NP)]  # firefly agents,
        self.Fireflies_tmp = [[np.random.rand() for i in range(self.D)] for j in range(
            self.NP)]  # intermediate pop
        self.Fitness = [0.0] * self.NP  # fitness values (Accuracy)
        self.I = [0.0] * self.NP  # light intensity
        self.nbest = [0.0] * self.NP  # the best solution found so far
        self.LB = 0  # lower bound
        self.UB = 1  # upper bound
        self.fbest = None  # the best
        self.evaluations = 0
        self.Fun = function

    def alpha_new(self, a):
        delta = 1.0 - math.pow((math.pow(10.0, -4.0) / 0.9), 1.0 / float(a))
        return (1 - delta) * self.alpha

    def sort_ffa(self):  # implementation of bubble sort

        for i in range(self.NP):
            self.Index[i] = i

        for i in range(0, (self.NP - 1)):
            j = i + 1
            for j in range(j, self.NP):
                if (self.I[i] > self.I[j]):
                    z = self.I[i]  # exchange attractiveness
                    self.I[i] = self.I[j]
                    self.I[j] = z
                    z = self.Fitness[i]  # exchange fitness
                    self.Fitness[i] = self.Fitness[j]
                    self.Fitness[j] = z
                    z = self.Index[i]  # exchange indexes
                    self.Index[i] = self.Index[j]
                    self.Index[j] = z


    def replace_ffa(self):  # replace the old population according to the new Index values
        # copy original population to a temporary area
        for i in range(self.NP):
            for j in range(self.D):
                self.Fireflies_tmp[i][j] = self.Fireflies[i][j]

        # generational selection in the sense of an EA
        for i in range(self.NP):
            for j in range(self.D):
                self.Fireflies[i][j] = self.Fireflies_tmp[self.Index[i]][j]

    def FindLimits(self, k):
        for i in range(self.D):
            if self.Fireflies[k][i] < self.LB:
                self.Fireflies[k][i] = self.LB
            if self.Fireflies[k][i] > self.UB:
                self.Fireflies[k][i] = self.UB

    def move_ffa(self):
        for i in range(self.NP):
            scale = abs(self.UB - self.LB)
            for j in range(self.NP):
                r = 0.0
                for k in range(self.D):
                    r += (self.Fireflies[i][k] - self.Fireflies[j][k]) * \
                        (self.Fireflies[i][k] - self.Fireflies[j][k])
                r = math.sqrt(r)
                if self.I[i] > self.I[j]:  # brighter and more attractive
                    beta0 = 1.0
                    beta = (beta0 - self.betamin) * math.exp(-self.gamma * math.pow(r, 2.0)) + self.betamin
                    for k in range(self.D):
                        r = random.uniform(0, 1)
                        tmpf = self.alpha * (r - 0.5) * scale
                        self.Fireflies[i][k] = self.Fireflies[i][
                            k] * (1.0 - beta) + self.Fireflies_tmp[j][k] * beta + tmpf
            self.FindLimits(i)

    def Run(self):
        print("Strated : ")
        while self.evaluations < self.nFES:
            print("in loop : ")
            # optional reducing of alpha
            #self.alpha = self.alpha_new(self.nFES/self.NP)
            self.evaluations = self.evaluations + 1
            # evaluate new solutions
            for i in range(self.NP):
                print("calculating fitness "+str(i))
                print(self.Fireflies[i])
                self.Fitness[i] = self.Fun(self.Fireflies[i])
                self.I[i] = self.Fitness[i]
            # ranking fireflies by their light intensit
            self.sort_ffa()
            # replace old population
            self.replace_ffa()
            # move all fireflies to the better locations
            self.move_ffa()
            #print(self.Fireflies[self.NP - 1])
        bestFirefly = self.Fireflies[self.NP - 1]
        return bestFirefly


def evaluation(feature_possibilities):
        feature_possibilities = np.round(feature_possibilities)

        feature_possibilities = feature_possibilities > np.float32(0.5)
        print(feature_possibilities)
        if sum(feature_possibilities)>2:

            selectedX = X[:, feature_possibilities]

            s = svm.SVC(kernel="poly", C=1)

            loocv = LeaveOneOut()
            evaluation = cross_val_score(s, selectedX, y,  cv=loocv)

            return evaluation.mean()
        else:
            return 0


In [4]:
import os
from PIL import Image
import numpy as np
# import keras
from tensorflow import keras 
import os
import cv2
from matplotlib import pyplot as plt
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.applications.vgg16 import decode_predictions
%pylab inline
import matplotlib.pyplot as plt

vggModel = VGG16(weights='imagenet', include_top=False)
resModel = ResNet50(weights='imagenet',include_top=False)
densModel = DenseNet121(weights='imagenet', include_top=False)

model = resModel

X = []
Y = []
classCount = 0

base_path='/content/drive/MyDrive/firefly/dataset_new'
a=0
source_path=base_path
for child in os.listdir(source_path):
    classCount +=1
    print(child)
    sub_path = os.path.join(source_path, child)
    if os.path.isdir(sub_path):
        for data_file in os.listdir(sub_path):
            Qry = Image.open(os.path.join(sub_path, data_file))
            Qry = Qry.convert("RGB")
            Qry_ = np.array(Qry.resize((100,100)))
            Qry = Qry_.reshape([-1,100,100,3])
            feature_2=(resModel.predict([Qry])).flatten()
            X.append(feature_2)
            Y.append(child)

print('No of classes : ',classCount)


Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


29097984/29084464 [==============================] - 0s 0us/step
class_3
class_1
class_2
No of classes :  3


In [5]:
X = np.array(X)
print(X.shape)

(3064, 32768)


In [6]:
y = np.array(Y)
print(y.shape)

(3064,)


In [7]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=10)
clf = clf.fit(X, Y)
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new.shape               


(3064, 4422)

In [ ]:
Algorithm = FireflyAlgorithm(evaluation)
Best = Algorithm.Run()


a = np.round(Best)

feature_take_or_not = a > np.float32(0.7)

# print(feature_take_or_not)

# print(Best)

true_number = np.array(np.unique(feature_take_or_not, return_counts=True)).T



# print(true_number)

# s = svm.SVC(kernel="linear")
# loocv = LeaveOneOut()
# evaluation = cross_val_score(s, bestX, y,  cv=loocv)
# print("Final Accuracy: %.6f%% (%.6f%%)" % (evaluation.mean(), evaluation.std() ))


Strated : 
in loop : 
calculating fitness 0
[0.9599633459225589, 0.11314513414812444, 0.9757614133641473, 0.7485615750209059, 0.346682423095486, 0.9753530517111284, 0.6256027695081159, 0.5377607120428278, 0.10317972219328997, 0.8482054512158479, 0.24095073637987563, 0.8181459839814247, 0.4833880287256681, 0.05948411932859465, 0.05937321582848165, 0.7962037372731144, 0.9257076270732829, 0.40461614971681614, 0.5401143303566794, 0.054070034245759846, 0.3100256669999508, 0.3606431710843083, 0.23749105223639444, 0.24903552174222243, 0.7423761667336891, 0.9451109564757515, 0.06141323373943708, 0.6951157866002262, 0.6484836136613816, 0.6358589400180075, 0.6840625306755889, 0.9012183251867003, 0.05066076322250279, 0.5393561315957209, 0.20217746286499638, 0.16284731444764955, 0.9789166347228215, 0.0814807270037109, 0.8070781573699094, 0.08329259545758494, 0.15291576191695433, 0.05805994773353784, 0.41749140355864633, 0.14616948971944954, 0.9325055515551163, 0.9321966712743429, 0.001082199139896

In [ ]:
bestX = X[:, feature_take_or_not]


In [ ]:
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
labelBinarizer = LabelEncoder()
y = labelBinarizer.fit_transform(Y)
        
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(bestX), np.array(y), 
                                                    test_size=0.2, random_state=42)

y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import pickle


clf1 =RandomForestClassifier(max_depth=2, random_state=0)

clf2 = GaussianNB()
clf3 = SVC(C=0.7, gamma='scale', kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('knn', clf1),('gnb',clf2),('svc',clf3)], voting='hard')
eclf.fit(X_train,y_train)

filename = 'finalized_model.sav'
pickle.dump(eclf, open(filename, 'wb'))

pickle.dump(labelBinarizer, open("labels.sav", 'wb'))
acc = eclf.score(X_test,y_test)
print(acc)